# Hydropandas Objects

In the HydroPandas Python package, the Obs and ObsCollection classes are designed to handle time series data related to hydrological observations.

The Obs class represents a single time series of measurements at a specific location, such as groundwater levels or precipitation amounts. It is a subclass of the pandas DataFrame, enriched with additional attributes and methods for the type of observation it holds. There are specialized subclasses of Obs for different measurement types, including:

- GroundwaterObs: for groundwater measurements
- WaterQualityObs: for (ground)water quality measurements
- WaterlvlObs: for surface water level measurements
- ModelObs: for observations from a MODFLOW model
- MeteoObs: for meteorological observations
- PrecipitationObs: for precipitation observations (subclass of MeteoObs)
- EvaporationObs: for evaporation observations (subclass of MeteoObs)

The ObsCollection class represents a collection of Obs objects, such as multiple groundwater level time series within a certain area. It is also a subclass of the pandas DataFrame, where each row contains metadata (e.g., coordinates of the observation point) and the corresponding Obs object that holds the measurements. Both Obs and ObsCollection classes include methods for reading data from various sources, facilitating the management and analysis of hydrological time series data.

## <a id=top></a>Notebook contents

1. [Obs](#Obs)
2. [ObsCollection](#ObsCollection)

In [ ]:
import numpy as np
import pandas as pd

import hydropandas as hpd

hpd.util.get_color_logger("INFO")

## Obs<a id=GroundwaterObs></a>

Creating an `Obs` object is very similar to creating a `DataFrame`. Below we create 3 differente Obs objects:

1. an empty Obs
2. an Obs with only metadata 
3. an Obs with metadata and measurements

In [ ]:
# 1. create an empty Obs object
o1 = hpd.Obs(name="my empty obs")
display(o1)

In [ ]:
# 2. create an Obs object with only metadata
o2 = hpd.Obs(
    name="my_observation",
    x=10,
    y=20,
    location="somewhere",
    filename="unknown",
    source="imagination",
    unit="m",
)
display(o2)

In [ ]:
# 3. create an Obs object with both metadata and measurements
meas_df = pd.DataFrame(
    index=pd.date_range(start="2020-01-01", periods=10, freq="D"),
    data={"value": np.random.rand(10)},
)
o3 = hpd.Obs(
    meas_df,
    name="smw",
    x=1000,
    y=22220,
    location="somewhere else",
    source="advanced imagination",
    unit="m",
)
display(o3)

#### Metadata

Access observation metadata as attributes.

In [ ]:
print(f"x coordinate of observation 1: {o1.x}")
print(f"x coordinate of observation 2: {o2.x}")
print(f"x coordinate of observation 3: {o3.x}")

In [ ]:
print(f"source of observation 1 is : {o1.source}")
print(f"location of observation 2 is : {o2.location}")
print(f"name of observation 3 is : {o3.name}")

#### Measurements

Access observation measurements as if the observation is a DataFrame with the measurements.

In [ ]:
display(o3["value"])  # show measurements

In [ ]:
perc85 = o3["value"].quantile(0.85)  # get percentile
print(f"the 85th percentile of my measurements is {perc85:.2f} {o3.unit}")

In [ ]:
o3["value"].plot(
    figsize=(7, 3),
    label=o3.name,
    ylabel=o3.unit,
    marker="o",
    legend=True,
    title="my observations",
);  # plot measurements

#### Obs types

Different Obs types have differente metadata. Groundwater observations have some extra properties `screen_top`, `screen_bottom`, `ground_level`, `tube_top` and `metadata_available`.

In [ ]:
gw_obs = hpd.GroundwaterObs(
    o3,
    name="smw_pb1",
    tube_nr=1,
    screen_top=-5,
    screen_bottom=-6,
    unit="m NAP",
    ground_level=3,
    tube_top=2.95,
    metadata_available=True,
)  # create a GroundwaterObs object from the Obs object
display(gw_obs)

### Modify

Sometimes you want to change measurement values or metadata of an Obs object.

In [ ]:
# modify measurements (similar to how you modify a pandas DataFrame)
o3.loc["2020-01-05":"2020-01-7", "value"] = 2  # set value of a specific date
o3.plot()

In [ ]:
# create new Obs object from existing
o4 = o3.copy()  # note use the copy method to create a new object
o4.loc["2020-01-05":"2020-01-7", "value"] = -1
o4.plot()

In [ ]:
# modify metadata by direct assignment
o4.name = "smw_modified"
o4.source = "smw"
display(o4)

In [ ]:
# modify metadata by direct assignment
o4.name = "smw_modified"
o4.source = "smw"
display(o4)

In [ ]:
# modify metadata by direct assignment
o4.name = "smw_modified"
o4.source = "smw"
display(o4)

### Additional metadata

You can have metadata for an observation that does not match any of the standard metadata names. For groundwater observations you may for example have the name of the company that constructed the measurement well. This additional metadata can be stored in the `meta` attribute as a dictionary. Below we create a GroundwaterObs object with this additional metadata and we print the metadata.

Note that `display(gw_obs)` will never display the meta dictionary with additional data.

In [ ]:
gw_obs = hpd.GroundwaterObs(
    o3,
    name="smw_pb1",
    tube_nr=1,
    screen_top=-5,
    screen_bottom=-6,
    unit="m NAP",
    ground_level=3,
    tube_top=2.95,
    metadata_available=True,
    meta={"contractor": "GeoDrill Inc."},
)
print(gw_obs.meta)

### Reading/writing Obs

Observations can be saved as a json, csv, excel or pickle file, see this table:

| type   | write function  | read function   | Human readable        | Store metadata | Write/read additional metadata*   | keep dtypes?    |
|--------|-----------------|-----------------|-----------------------|----------------|-----------------------------------|-----------------|
| json   | Obs.to_json     | hpd.read_json   | Yes                   | Yes            | Yes                               | Mostly          |
| csv    | Obs.to_csv      | Obs.from_csv    | Yes                   | Yes            | No                                | No              |
| pickle | Obs.to_pickle   | Obs.from_pickle | No                    | Yes            | Yes                               | Yes             |
| excel* | Obs.to_excel    | pd.read_excel   | Yes (in Excel)        | No             | No                                | No              |

*the to_excel method is the inherited method from a pandas DataFrame. The other methods are methods adapted for hydropandas.

##### json

Json is a human readable format that can be used to store observation objects. Additional metadata is kept in the json file and it is more robust for keeping the same dtypes. At times small details, such as the index frequency, may be different between the original file and the one that is written and read to a json file.

In [ ]:
# write to json
gw_obs.to_json("my_gw_obs.json", indent=4)
gw_obs

In [ ]:
# read from json
gw_obs_from_json = hpd.read_json("my_gw_obs.json")
gw_obs_from_json

In [ ]:
print("index frequency original:", gw_obs.index.freq)
print("index frequency from json:", gw_obs_from_json.index.freq)

##### csv

When Obs object are written and read from a csv file:
1. The datatypes may have changed
2. The additional metadata is lost.

In [ ]:
# save the groundwater observations to a csv file
gw_obs.to_csv("my_gw_obs.csv")

In [ ]:
# read the groundwater observations from a csv file
gw_obs_from_csv = hpd.GroundwaterObs.from_csv("my_gw_obs.csv")

In [ ]:
# 1. datatypes changed
print("datatype of gw_obs.screen_top:", type(gw_obs.screen_top))
print("datatype of gw_obs_from_csv.screen_top:", type(gw_obs_from_csv.screen_top))

# 2. additional metadata is not saved in the csv file
print(f"\nadditional metadata: {gw_obs.meta=}")
print(f"additional metadata: {gw_obs_from_csv.meta=}")

##### pickle
Pickle files are binary and not readable for humans. They are very fast to write/read and return an exact copy of the original file. Pickle files are not very useful for long-term storage because they:

- are only readable in Python
- are not stable across Python and package versions.
- contain references to exact class and module paths

In [ ]:
# save the object to a pickle file
gw_obs.to_pickle("my_gw_obs.pklz")

In [ ]:
# read the object from a pickle file
gw_obs2 = hpd.read_pickle("my_gw_obs.pklz")

In [ ]:
gw_obs2.equals(gw_obs)  # check if the two objects are equal

## ObsCollection<a id=ObsCollection></a>

An ObsCollection is a structured way to manage and analyse multiple time series of hydrological observations. It serves as a container for multiple Obs objects, which represent individual time series of measurements, such as groundwater levels, precipitation, or water quality.

Each row in an ObsCollection contains metadata (e.g., location, station name) and a corresponding Obs object holding the time series data. This structure allows for easy comparison, filtering, and statistical analysis across multiple observation sites.

In [ ]:
# create an empty ObsCollection
oc = hpd.ObsCollection()
print(oc)

In [ ]:
# create an ObsCollection with a single Obs object
oc = hpd.ObsCollection(o3)
oc

In [ ]:
# create an ObsCollection with multiple Obs objects
oc = hpd.ObsCollection([o1, o2, o3])
oc

#### ObsCollection metadata

Access the metadata using the standard DataFrame methods.

In [ ]:
print(f"the x coordinate of observation 2 is: {oc.loc['my_observation', 'x']}")
print(f"the location of observation 3 is: {oc.loc['smw', 'location']}")

#### ObsCollection observations

Access the Obs objects from the collection

In [ ]:
o3_1 = oc.loc["smw", "obs"]  # using the loc method
o3_2 = oc.get_obs("smw")  # using the get_obs method with the name
o3_3 = oc.get_obs(
    location="somewhere else"
)  # using the get_obs method with the location (only works if the location is unique)
id(o3_1) == id(o3_2) == id(o3_3)  # check if the three objects are the same

#### Slice ObsCollection

Filter and slice ObsCollections

In [ ]:
oc.loc[oc["y"] > 10]  # Selection based on the y coordinate

In [ ]:
oc.loc[oc["source"].str.contains("advanced")]  # Selection based on the location

### Modify ObsCollections

Below are some examples to modify ObsCollections. More details on merging observations and ObsCollections are available [here](04_merging_observations.ipynb).

- add an observation
- remove an observation
- modify metadata of an observation
- modify the timeseries of an observation
- add a copy of an existing observation
- replace an existing observation

#### Remove observation

Remove an observation from an ObsCollection using `drop`.

In [ ]:
# remove an observation
oc.drop("my_observation", inplace=True)

#### Add observation

Add an observation from an ObsCollection using `add_observation`.

Note: Adding an observation using `oc.loc[<name>,'obs'] = o` does not work and results in an empty 'obs' column

In [ ]:
# add an observation
oc.add_observation(o2, inplace=True)
oc

In [ ]:
# Do not add observations using loc!
oc_copy = oc.copy(deep=True)
oc_copy.loc["new_obs", "obs"] = gw_obs
oc_copy

#### Modify metadata
Use the `set_metadata_value` on the ObsCollection to modify metadata.

Note:  Metadata of a single observation is stored in two places: in the ObsCollection DataFrame and in the attribute of the Observation object. `set_metadata_value` will modify both which is preferred over setting the value only in the ObsCollection dataframe or only in the Observation attribute. The latter two will result in an inconsistent ObsCollection.



In [ ]:
# modify metadata of an observation
oc.set_metadata_value("my_observation", "x", 1815)
oc.set_metadata_value("my_observation", "y", 2025)
print(oc._is_consistent())  # check if the ObsCollection is consistent

In [ ]:
# Do not use this way to modify metadata!
oc_copy = oc.copy(deep=True)
oc_copy.loc["smw", "x"] = 100
oc_copy._is_consistent()  # check if the ObsCollection is consistent

#### Modify time series

There are several ways to modify time series of observations in a collection. Below we show a way using chained assignment and a two-step plan.

In [ ]:
# modify the timeseries of an observation

# 1 chained assignment
oc.loc["smw", "obs"].loc["2020-1-7":"2020-1-9", "value"] = 42
display(oc.loc["smw", "obs"])

# 2 two-step
o = oc.loc["smw", "obs"]
o.loc["2020-1-7":"2020-1-9", "value"] = 21
display(oc.loc["smw", "obs"])

#### Add observation copy

In [ ]:
# create a copy of an observation
o = oc.loc["my empty obs", "obs"].copy()

# modify some stuff
date_range = pd.date_range(start="2020-01-01", periods=10, freq="D")
o.index = date_range
o.loc[date_range, "value"] = np.arange(10)
o.name = "not so emtpy obs"

# add modified observation to collection
oc.add_observation(o, inplace=True)
oc

#### Replace observation

There are several ways to replace an observation in a collection:
1. remove the observation first and then add the observation that should replace the original.
2. use `add_observation` to add an observation with the same name as an existing observation in the collection. Hydropandas will try to merge the new observation with the existing observation.

In [ ]:
# create a copy of an observation
o = oc.loc["my empty obs", "obs"].copy()

# modify time series
date_range = pd.date_range(start="2020-01-01", periods=10, freq="D")
o.index = date_range
o.loc[date_range, "value"] = np.arange(10)

# replace existing observation
oc.add_observation(o, inplace=True)
oc

### Additional metadata

Additional metadata is not shown by default in an ObsCollection. It can be added manually by calling the `add_meta_to_df` method. By default all metadata is added but you can also specify a key from the meta dictionary to add.

In [ ]:
# create an Obs object with additional metadata
o2_with_meta = hpd.Obs(
    name="my_observation",
    x=10,
    y=20,
    location="somewhere",
    filename="unknown",
    source="imagination",
    unit="m",
    meta={"owner": "me", "project": "hydropandas"},
)

# create an ObsCollection with multiple Obs objects, one of them with additional metadata
oc = hpd.ObsCollection([o1, o2_with_meta, o3])

# metadata is not shown by default
oc

In [ ]:
# add metadata to the dataframe to show it
oc_with_meta = oc.add_meta_to_df()
oc_with_meta

### Read/write an ObsCollection

There are several options to write an ObsCollection to a file. The table below gives a broad overview on the options


| type   | write function          | read function  | Human readable        | Write/read additional metadata*   | keep dtypes?    |
|--------|-------------------------|----------------|-----------------------|-----------------------------------|-----------------|
| json   | ObsCollection.to_json   | hpd.read_json  | Yes                   | Yes                               | Mostly          |
| csv    | ObsCollection.to_csv    | hpd.read_csv   | Yes                   | No                                | No              |
| excel  | ObsCollection.to_excel  | hpd.read_excel | Yes (via excel)       | Only if exposed in oc**           | No              |
| pickle | ObsCollection.to_pickle | hpd.read_pickle| No                    | Yes                               | Yes             |

Writing to and reading from an excel, csv or json file slightly alters the properties of the ObsCollection, just like writing and reading a DataFrame to these formats would do. Reading/writing a pickle does not change anything.

##### json

In [ ]:
path = "my_obs_collection.json"
oc_with_meta.to_json(path, indent=4)
oc_with_meta

In [ ]:
oc_from_json = hpd.read_json(path)
oc_from_json

##### csv

In [ ]:
csvdir = "my_obs_collection"
oc_with_meta.to_csv(csvdir)
oc_with_meta

In [ ]:
oc_from_csv = hpd.read_csv(csvdir)  # read the ObsCollection from the csv files
oc_from_csv

##### excel

In [ ]:
oc_with_meta.to_excel("my_obs_collection.xlsx")  # write to excel

In [ ]:
# read excel file
oc_from_excel = hpd.read_excel("my_obs_collection.xlsx")
oc_from_excel

##### pickle

In [ ]:
oc_with_meta.to_pickle("my_obs_collection.pklz")  # write to pickle

In [ ]:
# read pickle
oc_from_pickle = hpd.read_pickle("my_obs_collection.pklz")
oc_from_pickle

### Extensions

To enhance the functionality of an ObsCollection, HydroPandas provides several extensions that add specialized methods for visualization, spatial analysis, and data processing. Some key extensions include:

- Plot Extension (ObsCollection.plot): Built-in plotting capabilities for visualizing time series data. Users can generate time series plots for individual or multiple observations, histograms, and other graphical representations to analyze trends and patterns in hydrological data.
- Geo Extension (ObsCollection.geo): Spatial analysis by integrating with geopandas. It allows users to obtain the extent of an ObsCollection, convert to another coordinate reference system and find nearby geometries.
- Groundwater Obs (ObsCollection.gwobs): Analyse and process groundwater observations. Users can find the REGIS layer of each tube and set the tube number based on the screen depth.
- Statistics (ObsCollection.stats): Statistical analysis of the observations. Users can obtain the number of consecutive years with more than 10 observations or find seasonal minimum and maximum values.




In [ ]:
oc.stats.get_first_last_obs_date()  # get the first and last observation date using the stats extension

In [ ]:
oc.geo.get_extent()  # get the extent of the observations using the geo extension